In [1]:
import numpy as np
import pysindy as ps
import timeit
from scipy.io import loadmat
from sklearn.metrics import mean_squared_error

In [2]:
dt=loadmat('lorenz_cos_0.000003.mat')['delta_t'][0][0]
x=loadmat('lorenz_cos_0.000003.mat')['tilde_u'][:,:,np.newaxis].transpose(1,0,2)
t=dt*np.arange(x.shape[1])
shape=np.concatenate([x.shape[:-1],[4]])
u=np.zeros(shape)
u[:,:,0]=np.cos(t)
u[:,:,1]=(np.roll(x,-1,axis=0))[:,:,0]#u_{j+1}
u[:,:,2]=(np.roll(x,2,axis=0))[:,:,0]
u[:,:,3]=(np.roll(x,1,axis=0))[:,:,0]
true_coefs=np.array([8,-1,-1,1,-1,1])
true_c = np.zeros([26,])
true_c[0] = true_coefs[0]
true_c[1] = true_coefs[1]
true_c[14] = true_coefs[2]
true_c[15] = true_coefs[3]
true_c[23] = true_coefs[4]
true_c[24] = true_coefs[5]

In [3]:
start=timeit.default_timer()
grid_shape=np.concatenate([x.shape[:-1],[2]])
spatiotemporal_grid=np.zeros(grid_shape)
spatiotemporal_grid[:,:,0]=np.arange(128)[:,np.newaxis]
spatiotemporal_grid[:,:,1]=t
np.random.seed(100)

In [4]:
weak_lib=ps.WeakPDELibrary(
    library_functions=[lambda x:x, lambda x,y: x*y, lambda x,y,z: x*y*z],
    function_names=[lambda x:x, lambda x,y:x+y,lambda x,y,z:x+y+z],
    derivative_order=0,
    spatiotemporal_grid=spatiotemporal_grid,
    K=5000,
    H_xt=[10,0.2/10],
    include_bias=True)

In [5]:
opt=ps.STLSQ(alpha=1e-8,threshold=0.3)
model = ps.SINDy(feature_library=weak_lib,optimizer=opt)
model.fit(x, t=dt, u=u)
stop=timeit.default_timer()
model.print(precision=5)

print("Weakform Runtime: ", stop-start)
print("Weakform Relative coefficient errors:",((model.optimizer.coef_[0][[0,1,14,15,23,24]]-true_coefs)**2/true_coefs**2)**0.5)
print("Total error: ",np.linalg.norm((model.optimizer.coef_[0]-true_c), ord=None, axis=None, keepdims=False)/np.linalg.norm(true_c))

(x0)' = 8.00000 1 + -0.99999 x0 + -1.00060 u1u3 + 1.00042 u2u3 + -0.99940 u0u1u3 + 0.99957 u0u2u3
Weakform Runtime:  4.084554700006265
Weakform Relative coefficient errors: [5.77274497e-07 8.21491678e-06 5.98124010e-04 4.22335576e-04
 6.02634376e-04 4.28749940e-04]
Total error:  0.00012529404172894536
